# 1 Setup

## 1.1 Import Modules

In [152]:
import arcpy
from os.path import join as JoinPath
from calendar import monthrange as MonthRange
from pandas import read_csv as ReadCsv
from pandas import DataFrame
from numpy import nan
from numpy import isnan
from tqdm import tqdm as Tqdm
from scipy import stats

## 1.2 Set Workplace and Environmental Variables

In [2]:
root = "D:/Repos/Air Pollution Exposure in Xi'an/Projects/Publish"

aprx = arcpy.mp.ArcGISProject('CURRENT')
map_ = aprx.listMaps('Map')[0]

arcpy.env.overwriteOutput = True

# 2 Data Processing

## 2.1 PM 2.5: Sites Data

### 2.1.1 Create a Feature Class of the Selected Sites (Points)

Table `sites_info.csv` records information of sites, with following structure:

| Column  | Type   | Meaning                                 |
| :------ | :----- | :-------------------------------------- |
| id      | String | Id of the site                          |
| name    | String | Name of the site                        |
| city    | String | Name of the city where the site locates |
| lng     | Double | Longitude of the site                   |
| lat     | Double | Latitude of the site                    |
| control | String | Omited                                  |

And table `selected_sites.csv` records the selected sites in this research,
with following structure:

| Column | Type   | Meaning        |
| :----- | :----- | :------------- |
| id     | String | Id of the site |

Firstly, define the paths of the tables:

In [ ]:
sitesInfoFilename = JoinPath(root, 'Data/PM25.Sites.China/sites_info.csv')
selectedSitesFilename = JoinPath(root, 'Data/PM25.Sites.China/selected_sites.csv')

Then, read the tables:

In [ ]:
sitesInfo = ReadCsv(sitesInfoFilename)
selectedSites = ReadCsv(selectedSitesFilename)

Method `DataFrame.join()` is used to filter the selected sites.

This method joins `DataFrame` by their indexes. Hence, set `id` as the indexes:

In [ ]:
sitesInfo.set_index('id', inplace = True)
selectedSites.set_index('id', inplace = True)

Apply left join by set the parameter `how` as `'left'`.

*Tips: left join refers to keep all records of the first data frame 
and drop unmatch records of the second data frame.*

In [ ]:
selectedSitesWithInfo = selectedSites.join(sitesInfo, how = 'left')

Reset index of the generated data frame. 
Otherwise, `id` cannot be gotten in the returned result of `DataFrame.to_dict('records')`.

In [ ]:
selectedSitesWithInfo.reset_index(inplace = True)

Now, we get the information of the selected sites. 
The next step is to create a feature class.

Define the path (referring to directory here), name, and filename of the feature class:

In [ ]:
featureClassPath = JoinPath(root, 'Publish.gdb')
featureClassName = 'SelectedSites'
featureClassFilename = JoinPath(featureClassPath, featureClassName)

Define the sptial reference of the feature class (CGS_WGS_84):

In [ ]:
spatialReference = arcpy.SpatialReference(4326)

Create a blank feature class to store the sites.
Set `geometry_type` to `'POINT'` and `spatial_reference` to CGS_WGS_84:

In [ ]:
arcpy.management.CreateFeatureclass(
    out_path = featureClassPath, 
    out_name = featureClassName, 
    geometry_type = 'POINT',
    spatial_reference = spatialReference
)

Add a field to store the `'id'`:

In [ ]:
arcpy.management.AddField(
    in_table = featureClassFilename, 
    field_name = 'id', 
    field_type = 'TEXT'
)

Iterate the selected sites, insert it into the feature class:

In [ ]:
with arcpy.da.InsertCursor(featureClassFilename, ['SHAPE@XY', 'id']) as cursor:
    for site in selectedSitesWithInfo.to_dict('records'):
        point = arcpy.Point(site['lng'], site['lat'])
        cursor.insertRow([point, site['id']])

Sometimes, ArcGIS Pro will raise error stating that the feature class is invaild.
By re-adding the feature class into the map, this error can be fit:

In [2]:
arcpy.management.Delete(
    in_data = featureClassName,
    data_type = ''
)

_ = map_.addDataFromPath(
    data_path = featureClassFilename,
    web_service_type = 'AUTOMATIC',
    custom_parameters = None
)

### 2.1.2 Estimate Daily PM 2.5 Concentration Using the Kriging Method

In [3]:
arcpy.management.CreateFileGDB(
    out_folder_path = root,
    out_name = 'Kriging',
    out_version = 'CURRENT'
)

selectedSitesLayerName = 'SelectedSites'
selectedSitesFilename = JoinPath(root, 'Publish.gdb', selectedSitesLayerName)

listOfSelectedSites = selectedSites.index
listOfColumns = ['hour']
listOfColumns.extend(listOfSelectedSites)

dailyDataTableNameFilename = JoinPath(root, 'pm25tmp.csv')
dailyDataTableColumns = ['id', 'wholeDay', 'daytime', 'nighttime']

with Tqdm(total = 365 * 3) as pbar:
    for month in range(1, 13):
        for day in range(1, MonthRange(2021, month)[1] + 1):
            dateString = '%s%s%s' % (2021, '%02d' % month, '%02d' % day)
            sitesDailyDataFilename = JoinPath(root, f'Data/PM25.Sites.China/china_sites_{dateString}.csv')
            hourlyData = ReadCsv(sitesDailyDataFilename)
            hourlyData = hourlyData[hourlyData['type'] == 'PM2.5'][listOfColumns]
            
            dailyData = DataFrame({
                'wholeDay': hourlyData.drop(columns = 'hour').mean(),
                'daytime': hourlyData[(hourlyData['hour'] >= 9) & (hourlyData['hour'] <= 18)].drop(columns = 'hour').mean(),
                'nighttime': hourlyData[hourlyData['hour'] <= 6].drop(columns = 'hour').mean()
            })
            dailyData.reset_index(inplace = True)
            dailyData.rename(
                columns = { 'index': 'id' },
                inplace = True
            )
            dailyData.to_csv(
                dailyDataTableNameFilename,
                index = False
            )
            
            arcpy.management.Delete(
                in_data = selectedSitesLayerName,
                data_type = ''
            )
            
            map_.addDataFromPath(
                data_path = selectedSitesFilename,
                web_service_type = 'AUTOMATIC',
                custom_parameters = None
            )
            
            arcpy.management.AddJoin(
                in_layer_or_view = selectedSitesLayerName, 
                in_field = 'id', 
                join_table = dailyDataTableNameFilename,
                join_field = 'id',
                join_type = 'KEEP_ALL',
                index_join_fields = 'NO_INDEX_JOIN_FIELDS'
            )

            for type_ in ['wholeDay', 'daytime', 'nighttime']:
                krigingOutputFilename = JoinPath(root, 'Kriging.gdb', f'DailyKriging_{dateString}_{type_}')
                try:
                    with arcpy.EnvManager(extent = '108.500670794889 34.0035907093902 109.508251052924 34.6026628845476'):
                        krigingOutput = arcpy.sa.Kriging(
                            selectedSitesLayerName, 
                            type_, 
                            'Spherical # # # #', 
                            0.00162879999999996,
                            'VARIABLE 12',
                            None
                        )
                        krigingOutput.save(krigingOutputFilename)
                except:
                    print(f'Empty: {dateString}.{type_}')
                pbar.update(1)

_ = arcpy.management.Delete(
    in_data = 'krigingOutput',
    data_type = ''
)

In [4]:
_ = arcpy.management.Delete(
    in_data = 'SelectedSites',
    data_type = ''
)

### 2.1.3 Calculate Monthly PM 2.5 Concentrations Using the Cell Statistics Tool, Based on Daily Concentrations

In [ ]:
with Tqdm(total = 12 * 3) as pbar:
    for month in range(1, 13):
        for type_ in ['wholeDay', 'daytime', 'nighttime']:
            listOfLayers = []
            for day in range(1, MonthRange(2021, month)[1] + 1):
                dateString = '%s%s%s' % (2021, '%02d' % month, '%02d' % day)
                krigingFilename = JoinPath(root, 'Kriging.gdb', f'DailyKriging_{dateString}_{type_}')
                if arcpy.Exists(krigingFilename):
                    listOfLayers.append(krigingFilename)

            krigingOutputFilename = JoinPath(root, 'Kriging.gdb', 'MonthlyKriging_%s_%s' % ('%02d' % month, type_))
            monthlyOutCellStatistics = arcpy.sa.CellStatistics(
                in_rasters_or_constants = listOfLayers, 
                statistics_type = 'MEAN', 
                ignore_nodata = 'DATA', 
                process_as_multiband = 'SINGLE_BAND',
                percentile_interpolation_type = 'AUTO_DETECT'
            )
            monthlyOutCellStatistics.save(krigingOutputFilename)
            
            pbar.update(1)

### 2.1.4 Calculate Quarterly PM 2.5 Concentrations Using the Cell Statistics Tool, Based on Monthly Concentrations

In [8]:
with Tqdm(total = 4 * 3) as pbar:
    for quarter in range(1, 5):
        for type_ in ['wholeDay', 'daytime', 'nighttime']:
            listOfLayers = []
            for month in range((quarter - 1) * 3 + 1, quarter * 3 + 1):
                krigingFilename = JoinPath(root, 'Kriging.gdb', 'MonthlyKriging_%s_%s' % ('%02d' % month, type_))
                listOfLayers.append(krigingFilename)

            krigingOutputFilename = JoinPath(root, 'Kriging.gdb', f'QuarterlyKriging_2021Q{quarter}_{type_}')
            quarterlyOutCellStatistics = arcpy.sa.CellStatistics(
                in_rasters_or_constants = listOfLayers, 
                statistics_type = 'MEAN', 
                ignore_nodata = 'DATA', 
                process_as_multiband = 'SINGLE_BAND',
                percentile_interpolation_type = 'AUTO_DETECT'
            )
            quarterlyOutCellStatistics.save(krigingOutputFilename)

            pbar.update(1)

100%|██████████| 12/12 [01:34<00:00,  7.88s/it]﻿


### 2.1.5 Calculate Annual PM 2.5 Concentrations Using the Cell Statistics Tool, Based on Quarterly Concentrations

In [6]:
for type_ in ['wholeDay', 'daytime', 'nighttime']:
    listOfLayers = []
    for quarter in range(1, 5):
        krigingFilename = JoinPath(root, 'Kriging.gdb', f'QuarterlyKriging_2021Q{quarter}_{type_}')
        listOfLayers.append(krigingFilename)

    krigingOutputFilename = JoinPath(root, 'Kriging.gdb', f'AnnualKriging_2021_{type_}')
    annualOutCellStatistics = arcpy.sa.CellStatistics(
        in_rasters_or_constants = listOfLayers, 
        statistics_type = 'MEAN', 
        ignore_nodata = 'DATA', 
        process_as_multiband = 'SINGLE_BAND',
        percentile_interpolation_type = 'AUTO_DETECT'
    )
    annualOutCellStatistics.save(krigingOutputFilename)

### 2.1.6 Remove the Temporal Layers

In [33]:
_ = arcpy.management.Delete(
    in_data = 'monthlyOutCellStatistics',
    data_type = ''
)

_ = arcpy.management.Delete(
    in_data = 'quarterlyOutCellStatistics',
    data_type = ''
)

_ = arcpy.management.Delete(
    in_data = 'annualOutCellStatistics',
    data_type = ''
)

## 2.2 PM 2.5: V5GL03

### 2.2.1 Create a File Geodatabase

In [ ]:
arcpy.management.CreateFileGDB(
    out_folder_path = root,
    out_name = 'V5GL03',
    out_version = 'CURRENT'
)

### 2.2.2 Monthly Data

In [41]:
for month in range(1, 13):
    monthString = '%02d' % month
    inNetCdfFile = JoinPath(root, f'Data/PM25.V5GL03.China/2021{monthString}.nc')
    outRasterLayerName = f'V5GL03_2021{monthString}'
    outRasterFilename = JoinPath(root, 'V5GL03.gdb', f'V5GL03_2021{monthString}')
    
    arcpy.md.MakeNetCDFRasterLayer(
        in_netCDF_file = inNetCdfFile,
        variable = 'GWRPM25',
        x_dimension = 'lon',
        y_dimension = 'lat',
        out_raster_layer = outRasterLayerName,
        band_dimension = '',
        dimension_values = None,
        value_selection_method = 'BY_VALUE',
        cell_registration = 'CENTER'
    )
    
    arcpy.management.CopyRaster(
        in_raster = outRasterLayerName,
        out_rasterdataset = outRasterFilename
    )

In [43]:
for month in range(1, 13):
    monthString = '%02d' % month
    rasterLayerName = f'V5GL03_2021{monthString}'
    arcpy.management.Delete(
        in_data = rasterLayerName,
        data_type = ''
    )
    arcpy.management.Delete(
        in_data = rasterLayerName,
        data_type = ''
    )

### 2.2.3 Quarterly Data

In [116]:
with Tqdm(total = 4) as pbar:
    for quarter in range(1, 5):
        list_of_layers = []
        for month in range((quarter - 1) * 3 + 1, quarter * 3 + 1):
            month_string = '%02d' % month
            monthly_data = JoinPath(root, 'V5GL03.gdb', f'V5GL03_2021{month_string}')
            list_of_layers.append(monthly_data)

        quarterly_data_filename = JoinPath(root, 'V5GL03.gdb', f'V5GL03_2021Q{quarter}')
        quarterly_data = arcpy.sa.CellStatistics(
            in_rasters_or_constants = list_of_layers, 
            statistics_type = 'MEAN', 
            ignore_nodata = 'DATA', 
            process_as_multiband = 'SINGLE_BAND',
            percentile_interpolation_type = 'AUTO_DETECT'
        )
        quarterly_data.save(quarterly_data_filename)

        pbar.update(1)

100%|██████████| 4/4 [00:21<00:00,  5.27s/it]﻿


## 2.3 Mobile Photo Data

In [17]:
layers = [
    '2021-03-17', '2021-03-20', '2021-06-16', '2021-06-19',
    '2021-09-15', '2021-09-18', '2021-11-24', '2021-11-27'
]

In [45]:
listOfGridIdsFilename = JoinPath(root, 'Data/MobilePhoto.Xian/GridIdsList.txt')
gridsShapeFilename = JoinPath(root, 'Data/MobilePhoto.Xian', layers[0])

featureClassPath = JoinPath(root, 'Publish.gdb')
featureClassName = 'GridShape'
featureClassFilename = JoinPath(featureClassPath, featureClassName)

spatialReference = arcpy.SpatialReference(4326)

arcpy.management.CreateFeatureclass(
    out_path = featureClassPath, 
    out_name = featureClassName, 
    geometry_type = 'POLYGON',
    spatial_reference = spatialReference
)

arcpy.management.AddField(featureClassFilename, 'GridID', 'LONG')

with arcpy.da.InsertCursor(featureClassFilename, ['SHAPE@', 'GridID']) as cursor:
    with open(listOfGridIdsFilename, 'r') as f:
        listOfGridIds = [int(id_.rstrip()) for id_ in f.readlines()]
        
    with open(gridsShapeFilename, 'r', encoding = 'utf-8') as f:
        rawRows = f.read().rstrip().split('\n')

    for row in rawRows:
        elements = row.split(';')
        grid = dict(zip(keys, elements))

        grid['GridID'] = int(grid['GridID'])
        
        if grid['GridID'] in listOfGridIds:
            points = []
            pointsString = grid['Geometry'].split(',')
            for pointString in pointsString:
                coordsString = pointString.split(' ')
                points.append(arcpy.Point(float(coordsString[0]), float(coordsString[1])))
            
            polygon = arcpy.Polygon(arcpy.Array(points), spatialReference)
            cursor.insertRow([polygon, grid['GridID']])

arcpy.management.Delete(
    in_data = featureClassName,
    data_type = ''
)

_ = map_.addDataFromPath(
    data_path = featureClassFilename,
    web_service_type = 'AUTOMATIC',
    custom_parameters = None
)

In [18]:
keys = [
    'GridID', 'Geometry', 'CenterPoint',
    'PopulationNighttime', 'PopulationDaytime', 'PropOutProv', 'PropOutCity'
    'City', 'County'
]

list_of_grid_ids_filename = JoinPath(root, 'Data/MobilePhoto.Xian/GridIdsList.txt')

with open(list_of_grid_ids_filename, 'r') as f:
    list_of_grid_ids = [int(id_.rstrip()) for id_ in f.readlines()]

non_local_data = {}
for quarter in range(1, 5):
    weekday_layer_id = (quarter - 1) * 2
    weekend_layer_id = (quarter - 1) * 2 + 1
    
    weekday_layer = layers[weekday_layer_id]
    weekend_layer = layers[weekend_layer_id]
    
    weekday_layer_filename = JoinPath(root, 'Data/MobilePhoto.Xian', weekday_layer)
    weekend_layer_filename = JoinPath(root, 'Data/MobilePhoto.Xian', weekend_layer)
    
    data = {}
    
    with open(weekday_layer_filename, 'r', encoding = 'utf-8') as f:
        weekday_raw_rows = f.read().rstrip().split('\n')

    for row in weekday_raw_rows:
        elements = row.split(';')
        grid = dict(zip(keys, elements))

        grid['GridID'] = int(grid['GridID'])

        if grid['GridID'] in list_of_grid_ids:
            non_local_weekday = float(grid['PropOutCity']) if grid['PropOutCity'] != '' else nan
            data.update({
                grid['GridID']: {
                    'NonLocalWeekday': non_local_weekday
                }
            })
    
    with open(weekend_layer_filename, 'r', encoding = 'utf-8') as f:
        weekend_raw_rows = f.read().rstrip().split('\n')
    
    for row in weekend_raw_rows:
        elements = row.split(';')
        grid = dict(zip(keys, elements))

        grid['GridID'] = int(grid['GridID'])

        if grid['GridID'] in list_of_grid_ids:
            non_local_weekday = data[grid['GridID']]['NonLocalWeekday']
            non_local_weekend = float(grid['PropOutCity']) if grid['PropOutCity'] != '' else nan
            
            if (not isnan(non_local_weekday)) and (not isnan(non_local_weekend)):
                non_local = non_local_weekend * 2 / 7 + non_local_weekday * 5 / 7
            elif (not isnan(non_local_weekday)) and isnan(non_local_weekend):
                non_local = non_local_weekday
            elif isnan(non_local_weekday) and (not isnan(non_local_weekend)):
                non_local = non_local_weekend
            else:
                non_local = nan
            
            data[grid['GridID']].update({
                'NonLocalWeekend': non_local_weekend,
                'NonLocal': non_local
            })
    
    non_local_data.update({
        f'Q{quarter}': data
    })

In [19]:
for quarter in range(1, 5):
    non_local_data_section = non_local_data[f'Q{quarter}']

    table_out_path = JoinPath(root, 'Social.gdb')
    table_out_name = f'DailyMobilePhone_2021Q{quarter}'
    table_filename = JoinPath(table_out_path, table_out_name)
    
    arcpy.management.CreateTable(
        out_path = table_out_path,
        out_name = table_out_name,
        template = None,
        config_keyword = '',
        out_alias = ''
    )

    arcpy.management.AddField(table_filename, 'GridID', 'LONG')
    arcpy.management.AddField(table_filename, 'NonLocal', 'DOUBLE')
    
    with arcpy.da.InsertCursor(table_filename, ['GridID', 'NonLocal']) as cursor:
        for grid_id in non_local_data_section:
            cursor.insertRow([grid_id, non_local_data_section[grid_id]['NonLocal']])
    
    arcpy.management.Delete(
        in_data = table_out_name,
        data_type = ''
    )

## 2.4 House Price Data

In [32]:
in_features = JoinPath(root, 'Data/MainUrbanArea.Xian/GridShapeGaode.shp')
out_path = JoinPath(root, 'Social.gdb')
out_name = 'GridShapeGaode'

with arcpy.EnvManager(addOutputsToMap = False):
    arcpy.conversion.FeatureClassToFeatureClass(
        in_features = in_features,
        out_path = out_path,
        out_name = out_name
    )

In [25]:
arcpy.management.CreateFileGDB(
    out_folder_path = root,
    out_name = 'Social',
    out_version = 'CURRENT'
)

<Result "D:/Users/PiesPro/OneDrive/Workspace/Air Pollution Exposure in Xi'an/Projects/Publish\\Social.gdb">

In [28]:
communities_house_price_filename = JoinPath(root, 'Data/HousePrice.Xian/Communities.20230311.Gaode.csv')

feature_class_path = JoinPath(root, 'Social.gdb')
feature_class_name = 'CommunitiesHousePrice'
feature_class_filename = JoinPath(feature_class_path, feature_class_name)

spatial_reference = arcpy.SpatialReference(4326)

arcpy.management.CreateFeatureclass(
    out_path = feature_class_path, 
    out_name = feature_class_name, 
    geometry_type = 'POINT',
    spatial_reference = spatial_reference
)

arcpy.management.AddField(feature_class_filename, 'AvgUnitPrice', 'LONG')
arcpy.management.AddField(feature_class_filename, 'Count', 'LONG')

with arcpy.da.InsertCursor(feature_class_filename, ['SHAPE@XY', 'AvgUnitPrice', 'Count']) as cursor:
    communities_house_price = \
        ReadCsv(communities_house_price_filename, encoding = 'utf-8').\
        to_dict('records')
    
    for community in communities_house_price:
        point = arcpy.Point(community['Longitude'], community['Latitude'])
        cursor.insertRow([point, community['AvgUnitPrice'], community['Count']])

arcpy.management.Delete(
    in_data = feature_class_name,
    data_type = ''
)

<Result 'true'>

In [29]:
in_json_file = JoinPath(root, 'Data/HousePrice.Xian/Bizcircles.20230311.Gaode.json')
out_features = JoinPath(root, 'Social.gdb/Bizcircles')

with arcpy.EnvManager(addOutputsToMap = False):
    arcpy.conversion.JSONToFeatures(
        in_json_file = in_json_file,
        out_features = out_features,
        geometry_type = 'POLYGON'
    )

<Result "D:\\Users\\PiesPro\\OneDrive\\Workspace\\Air Pollution Exposure in Xi'an\\Projects\\Publish\\Social.gdb\\Bizcircles">

In [30]:
in_features = JoinPath(root, 'Social.gdb/CommunitiesHousePrice')
out_raster = JoinPath(root, 'Social.gdb/HousePriceSurface')
in_barrier_features = JoinPath(root, 'Social.gdb/Bizcircles')

arcpy.ga.DiffusionInterpolationWithBarriers(
    in_features = in_features,
    z_field = 'AvgUnitPrice',
    out_raster = out_raster,
    in_barrier_features = in_barrier_features,
    bandwidth = None,
    number_iterations = 100,
    weight_field = 'Count',
    in_additive_barrier_raster = None,
    in_cumulative_barrier_raster = None,
    in_flow_barrier_raster = None
)

<Result ''>

In [33]:
in_zone_data = JoinPath(root, 'Social.gdb/GridShapeGaode')
in_value_raster = JoinPath(root, 'Social.gdb/HousePriceSurface')
out_table = JoinPath(root, 'Social.gdb/ZonalSt_HousePrice')

with arcpy.EnvManager(addOutputsToMap = False):
    arcpy.ia.ZonalStatisticsAsTable(
        in_zone_data = in_zone_data,
        zone_field = 'GridID',
        in_value_raster = in_value_raster,
        out_table = out_table,
        ignore_nodata = 'DATA',
        statistics_type = 'MEAN',
        process_as_multidimensional = 'CURRENT_SLICE',
        percentile_values = 90,
        percentile_interpolation_type = 'AUTO_DETECT',
        circular_calculation = 'ARITHMETIC',
        circular_wrap_value = 360
    )

## 2.5 Pollution in Grids

In [47]:
for quarter in range(1, 5):
    for type_ in ['daytime', 'nighttime']:
        in_zone_data = JoinPath(root, 'Publish.gdb', 'GridShape')
        in_value_raster = JoinPath(root, 'Kriging.gdb', f'QuarterlyKriging_2021Q{quarter}_{type_}')
        out_table = JoinPath(root, 'Kriging.gdb', f'ZonalSt_QuarterlyKriging_2021Q{quarter}_{type_}')

        with arcpy.EnvManager(addOutputsToMap = False):
            arcpy.ia.ZonalStatisticsAsTable(
                in_zone_data = in_zone_data,
                zone_field = 'GridID',
                in_value_raster = in_value_raster,
                out_table = out_table,
                ignore_nodata = 'DATA',
                statistics_type = 'MEAN',
                process_as_multidimensional = 'CURRENT_SLICE',
                percentile_values = 90,
                percentile_interpolation_type = 'AUTO_DETECT',
                circular_calculation = 'ARITHMETIC',
                circular_wrap_value = 360
            )

## 2.6 GWR Quarterly Datasets

In [20]:
arcpy.management.CreateFileGDB(
    out_folder_path = root,
    out_name = 'Gwr',
    out_version = 'CURRENT'
)

<Result "D:/Users/PiesPro/OneDrive/Workspace/Air Pollution Exposure in Xi'an/Projects/Publish\\Gwr.gdb">

In [21]:
grid_shape_layer_name = 'GridShape'
house_price_layer_name = 'ZonalSt_HousePrice'
grid_shape_filename = JoinPath(root, 'Publish.gdb', grid_shape_layer_name)
house_price_filename = JoinPath(root, 'Social.gdb', house_price_layer_name)

map_.addDataFromPath(
    data_path = grid_shape_filename,
    web_service_type = 'AUTOMATIC',
    custom_parameters = None
)
map_.addDataFromPath(
    data_path = house_price_filename,
    web_service_type = 'AUTOMATIC',
    custom_parameters = None
)

for quarter in range(1, 5):
    mobile_phone_layer_name = f'DailyMobilePhone_2021Q{quarter}'
    pm25_daytime_layer_name = f'ZonalSt_QuarterlyKriging_2021Q{quarter}_daytime'
    pm25_nighttime_layer_name = f'ZonalSt_QuarterlyKriging_2021Q{quarter}_nighttime'
    
    mobile_phone_filename = JoinPath(root, 'Social.gdb', mobile_phone_layer_name)
    pm25_daytime_filename = JoinPath(root, 'Kriging.gdb', pm25_daytime_layer_name)
    pm25_nighttime_filename = JoinPath(root, 'Kriging.gdb', pm25_nighttime_layer_name)

    map_.addDataFromPath(
        data_path = mobile_phone_filename,
        web_service_type = 'AUTOMATIC',
        custom_parameters = None
    )
    map_.addDataFromPath(
        data_path = pm25_daytime_filename,
        web_service_type = 'AUTOMATIC',
        custom_parameters = None
    )
    map_.addDataFromPath(
        data_path = pm25_nighttime_filename,
        web_service_type = 'AUTOMATIC',
        custom_parameters = None
    )
    
    arcpy.management.AddJoin(
        in_layer_or_view = grid_shape_layer_name,
        in_field = 'GridID',
        join_table = house_price_layer_name,
        join_field = 'GridID',
        join_type = 'KEEP_ALL',
        index_join_fields = 'NO_INDEX_JOIN_FIELDS'
    )
    arcpy.management.AddJoin(
        in_layer_or_view = grid_shape_layer_name,
        in_field = f'{grid_shape_layer_name}.GridID',
        join_table = mobile_phone_layer_name,
        join_field = 'GridID',
        join_type = 'KEEP_ALL',
        index_join_fields = 'NO_INDEX_JOIN_FIELDS'
    )
    arcpy.management.AddJoin(
        in_layer_or_view = grid_shape_layer_name,
        in_field = f'{grid_shape_layer_name}.GridID',
        join_table = pm25_daytime_layer_name,
        join_field = 'GridID',
        join_type = 'KEEP_ALL',
        index_join_fields = 'NO_INDEX_JOIN_FIELDS'
    )
    arcpy.management.AddJoin(
        in_layer_or_view = grid_shape_layer_name,
        in_field = f'{grid_shape_layer_name}.GridID',
        join_table = pm25_nighttime_layer_name,
        join_field = 'GridID',
        join_type = 'KEEP_ALL',
        index_join_fields = 'NO_INDEX_JOIN_FIELDS'
    )
    
    out_features = JoinPath(root, 'Gwr.gdb', f'Gwr_2021Q{quarter}')
    field_mapping = f'GridID "GridID" true true false 4 Long 0 0,First,#,GridShape,{grid_shape_layer_name}.GridID,-1,-1;' \
        f'HousePrice "HousePrice" true true false 8 Double 0 0,First,#,GridShape,{house_price_layer_name}.MEAN,-1,-1;' \
        f'NonLocal "NonLocal" true true false 8 Double 0 0,First,#,GridShape,{mobile_phone_layer_name}.NonLocal,-1,-1;' \
        f'PolDay "PolDay" true true false 8 Double 0 0,First,#,GridShape,{pm25_daytime_layer_name}.MEAN,-1,-1;' \
        f'PolNight "PolNight" true true false 8 Double 0 0,First,#,GridShape,{pm25_nighttime_layer_name}.MEAN,-1,-1'
    arcpy.conversion.ExportFeatures(
        in_features = grid_shape_layer_name,
        out_features = out_features,
        use_field_alias_as_name = 'NOT_USE_ALIAS',
        field_mapping = field_mapping,
        sort_field = None
    )
    
    arcpy.management.RemoveJoin(
        in_layer_or_view = grid_shape_layer_name,
        join_name = ''
    )

## 2.7 Data Validation

In [23]:
in_features = JoinPath(root, 'Data/MainUrbanArea.Xian/MainUrbanArea.shp')
out_features = JoinPath(root, 'Publish.gdb/MainUrbanArea')

with arcpy.EnvManager(addOutputsToMap = False):
    arcpy.conversion.ExportFeatures(
        in_features = in_features,
        out_features = out_features
    )

<Result "D:\\Users\\PiesPro\\OneDrive\\Workspace\\Air Pollution Exposure in Xi'an\\Projects\\Publish\\Publish.gdb\\MainUrbanArea">

In [18]:
arcpy.management.CreateFileGDB(
    out_folder_path = root,
    out_name = 'Validation',
    out_version = 'CURRENT'
)

<Result "D:/Users/PiesPro/OneDrive/Workspace/Air Pollution Exposure in Xi'an/Projects/Publish\\Validation.gdb">

In [20]:
for month in range(1, 13):
    monthString = '%02d' % month
    
    kringingFullName = JoinPath(root, 'Kriging.gdb', f'MonthlyKriging_{monthString}_wholeDay')
    v5gl03FullName = JoinPath(root, 'V5GL03.gdb', f'V5GL03_2021{monthString}')
    
    outputRasterPath = JoinPath(root, 'Validation.gdb', f'Validation_2021{monthString}')

    output_raster = arcpy.ia.RasterCalculator(
        rasters = [kringingFullName, v5gl03FullName], 
        input_names = ['kringing', 'v5gl03'],
        expression = '(kringing - v5gl03) / kringing'
    )
    output_raster.save(outputRasterPath)

In [119]:
for quarter in range(1, 5):
    kringing_filename = JoinPath(root, 'Kriging.gdb', f'QuarterlyKriging_2021Q{quarter}_wholeDay')
    v5gl03_filename = JoinPath(root, 'V5GL03.gdb', f'V5GL03_2021Q{quarter}')
    
    output_raster_path = JoinPath(root, 'Validation.gdb', f'Validation_2021Q{quarter}')

    output_raster = arcpy.ia.RasterCalculator(
        rasters = [kringing_filename, v5gl03_filename], 
        input_names = ['kringing', 'v5gl03'],
        expression = '(kringing - v5gl03) / kringing'
    )
    output_raster.save(output_raster_path)

In [117]:
#=====================#

In [127]:
for quarter in range(1, 5):
    in_zone_data = JoinPath(root, 'Publish.gdb', 'GridShape')
    in_value_raster = JoinPath(root, 'V5GL03.gdb', f'V5GL03_2021Q{quarter}')
    out_table = JoinPath(root, 'Validation.gdb', f'ZonalSt_Validation_2021Q{quarter}')

    with arcpy.EnvManager(addOutputsToMap = False):
        arcpy.ia.ZonalStatisticsAsTable(
            in_zone_data = in_zone_data,
            zone_field = 'GridID',
            in_value_raster = in_value_raster,
            out_table = out_table,
            ignore_nodata = 'DATA',
            statistics_type = 'MEAN',
            process_as_multidimensional = 'CURRENT_SLICE',
            percentile_values = 90,
            percentile_interpolation_type = 'AUTO_DETECT',
            circular_calculation = 'ARITHMETIC',
            circular_wrap_value = 360
        )

In [130]:
for quarter in range(1, 5):
    in_zone_data = JoinPath(root, 'Publish.gdb', 'GridShape')
    in_value_raster = JoinPath(root, 'Kriging.gdb', f'QuarterlyKriging_2021Q{quarter}_wholeDay')
    out_table = JoinPath(root, 'Validation.gdb', f'ZonalSt_Original_2021Q{quarter}')

    with arcpy.EnvManager(addOutputsToMap = False):
        arcpy.ia.ZonalStatisticsAsTable(
            in_zone_data = in_zone_data,
            zone_field = 'GridID',
            in_value_raster = in_value_raster,
            out_table = out_table,
            ignore_nodata = 'DATA',
            statistics_type = 'MEAN',
            process_as_multidimensional = 'CURRENT_SLICE',
            percentile_values = 90,
            percentile_interpolation_type = 'AUTO_DETECT',
            circular_calculation = 'ARITHMETIC',
            circular_wrap_value = 360
        )

In [138]:
quarterly_validation_data = []
for quarter in range(1, 5):
    validation_table = JoinPath(root, 'Validation.gdb', f'ZonalSt_Validation_2021Q{quarter}')
    original_table = JoinPath(root, 'Validation.gdb', f'ZonalSt_Original_2021Q{quarter}')

    cursor =  arcpy.SearchCursor(validation_table)
    for row in cursor:
        quarterly_validation_data.append({
            'Quarter': quarter,
            'GridID': row.getValue('GridID'),
            'Type': 'Validation',
            'Value': row.getValue('MEAN')
        })

    cursor = arcpy.SearchCursor(original_table)
    for row in cursor:
        quarterly_validation_data.append({
            'Quarter': quarter,
            'GridID': row.getValue('GridID'),
            'Type': 'Original',
            'Value': row.getValue('MEAN')
        })

In [159]:
quarterly_validation_dataframe = DataFrame(quarterly_validation_data).\
    pivot(index = ['Quarter', 'GridID'], columns = ['Type'], values = 'Value').\
    reset_index()

correlation_results = {}
for quarter in range(1, 5):
    subset_dataframe = quarterly_validation_dataframe.query(f'Quarter == {quarter} and Original == Original and Validation == Validation')
    correlation_results[quarter] = stats.pearsonr(subset_dataframe['Original'], subset_dataframe['Validation'])
    
correlation_results

{1: (0.6546004688549981, 1.8032854618734685e-106), 2: (0.7106062895950966, 2.6928426121935606e-133), 3: (0.4486193025295767, 7.315805192476115e-44), 4: (0.8004789349038788, 3.677223913366411e-193)}

## 2.8 For Visualization

### 2.8.1 Concentration Maps

In [5]:
arcpy.management.CreateFileGDB(
    out_folder_path = root,
    out_name = 'Plot',
    out_version = 'CURRENT'
)

<Result "D:/Users/PiesPro/OneDrive/Workspace/Air Pollution Exposure in Xi'an/Projects/Publish\\Plot.gdb">

In [21]:
with Tqdm(total = 4 * 3) as pbar:
    for quarter in range(1, 5):
        for type_ in ['wholeDay', 'daytime', 'nighttime']:
            in_raster = JoinPath(root, 'Kriging.gdb', f'QuarterlyKriging_2021Q{quarter}_{type_}')
            in_mask_data = JoinPath(root, 'Publish.gdb', 'MainUrbanArea')
            out_raster_name = JoinPath(root, 'Plot.gdb', f'PM25_2021Q{quarter}_{type_}')

            out_raster = arcpy.sa.ExtractByMask(
                in_raster = in_raster,
                in_mask_data = in_mask_data,
                extraction_area = 'INSIDE'
            )
            out_raster.save(out_raster_name)

            pbar.update(1)

100%|██████████| 12/12 [01:05<00:00,  5.48s/it]﻿


In [22]:
for type_ in ['wholeDay', 'daytime', 'nighttime']:
    in_raster = JoinPath(root, 'Kriging.gdb', f'AnnualKriging_2021_{type_}')
    in_mask_data = JoinPath(root, 'Publish.gdb', 'MainUrbanArea')
    out_raster_name = JoinPath(root, 'Plot.gdb', f'PM25_2021_{type_}')

    out_raster = arcpy.sa.ExtractByMask(
        in_raster = in_raster,
        in_mask_data = in_mask_data,
        extraction_area = 'INSIDE'
    )
    out_raster.save(out_raster_name)

### 2.8.2 Validation

In [111]:
for month in range(1, 13):
    month_string = '%02d' % month
    in_raster = JoinPath(root, 'Validation.gdb', f'Validation_2021{month_string}')
    in_mask_data = JoinPath(root, 'Publish.gdb', 'MainUrbanArea')
    out_raster_name = JoinPath(root, 'Plot.gdb', f'Validation_2021{month_string}')

    out_raster = arcpy.sa.ExtractByMask(
        in_raster = in_raster,
        in_mask_data = in_mask_data,
        extraction_area = 'INSIDE'
    )
    out_raster.save(out_raster_name)

In [120]:
for quarter in range(1, 5):
    in_raster = JoinPath(root, 'Validation.gdb', f'Validation_2021Q{quarter}')
    in_mask_data = JoinPath(root, 'Publish.gdb', 'MainUrbanArea')
    out_raster_name = JoinPath(root, 'Plot.gdb', f'Validation_2021Q{quarter}')

    out_raster = arcpy.sa.ExtractByMask(
        in_raster = in_raster,
        in_mask_data = in_mask_data,
        extraction_area = 'INSIDE'
    )
    out_raster.save(out_raster_name)

# 3 Moran's I

## 3.1 Global Moran's I

In [69]:
global_morans_i = {}
for quarter in range(1, 5):
    input_feature_class = JoinPath(root, 'Gwr.gdb', f'Gwr_2021Q{quarter}')
    morans_i = arcpy.stats.SpatialAutocorrelation(
        Input_Feature_Class = input_feature_class,
        Input_Field = 'NonLocal',
        Generate_Report = None,
        Conceptualization_of_Spatial_Relationships = 'FIXED_DISTANCE_BAND',
        Distance_Method = 'EUCLIDEAN_DISTANCE',
        Standardization = 'ROW',
        Distance_Band_or_Threshold_Distance = 2341,
        Weights_Matrix_File = None,
        number_of_neighbors = None
    )
    global_morans_i[f'NonLocal.Q{quarter}'] = morans_i

In [70]:
feature_name = JoinPath(root, 'Gwr.gdb', f'Gwr_2021Q1')
morans_i = arcpy.stats.SpatialAutocorrelation(
    Input_Feature_Class = feature_name,
    Input_Field = 'HousePrice',
    Generate_Report = None,
    Conceptualization_of_Spatial_Relationships = 'FIXED_DISTANCE_BAND',
    Distance_Method = 'EUCLIDEAN_DISTANCE',
    Standardization = 'ROW',
    Distance_Band_or_Threshold_Distance = 2341,
    Weights_Matrix_File = None,
    number_of_neighbors = None
)
global_morans_i['HousePrice'] = morans_i

In [105]:
DataFrame([
    {
        'Label': i,
        'I': global_morans_i[i][0],
        'z-score': global_morans_i[i][1],
        'p-value': global_morans_i[i][2]
    }
    for i in global_morans_i
])

,Label,I,z-score,p-value
0,NonLocal.Q1,0.599989,50.442767,0
1,NonLocal.Q2,0.628182,52.483682,0
2,NonLocal.Q3,0.729686,65.471711,0
3,NonLocal.Q4,0.650095,58.251977,0
4,HousePrice,0.75071,62.664635,0


## 3.2 Local Moran's I

In [72]:
for quarter in range(1, 5):
    input_feature_class = JoinPath(root, 'Gwr.gdb', f'Gwr_2021Q{quarter}')
    output_feature_class = JoinPath(root, 'Gwr.gdb', f'LocalMoransI_2021Q{quarter}_NonLocal')
    arcpy.stats.ClustersOutliers(
        Input_Feature_Class = input_feature_class,
        Input_Field = 'NonLocal',
        Output_Feature_Class = output_feature_class,
        Conceptualization_of_Spatial_Relationships = 'FIXED_DISTANCE_BAND',
        Distance_Method = 'EUCLIDEAN_DISTANCE',
        Standardization = 'ROW',
        Distance_Band_or_Threshold_Distance = 2341,
        Weights_Matrix_File = None,
        Apply_False_Discovery_Rate__FDR__Correction = 'APPLY_FDR',
        Number_of_Permutations = 9999,
        number_of_neighbors = None
    )

In [73]:
input_feature_class = JoinPath(root, 'Gwr.gdb', f'Gwr_2021Q1')
output_feature_class = JoinPath(root, 'Gwr.gdb', f'LocalMoransI_2021_HousePrice')
arcpy.stats.ClustersOutliers(
    Input_Feature_Class = input_feature_class,
    Input_Field = 'HousePrice',
    Output_Feature_Class = output_feature_class,
    Conceptualization_of_Spatial_Relationships = 'FIXED_DISTANCE_BAND',
    Distance_Method = 'EUCLIDEAN_DISTANCE',
    Standardization = 'ROW',
    Distance_Band_or_Threshold_Distance = 2341,
    Weights_Matrix_File = None,
    Apply_False_Discovery_Rate__FDR__Correction = 'APPLY_FDR',
    Number_of_Permutations = 9999,
    number_of_neighbors = None
)

<Result "D:\\Users\\PiesPro\\OneDrive\\Workspace\\Air Pollution Exposure in Xi'an\\Projects\\Publish\\Gwr.gdb\\LocalMoransI_2021_HousePrice">

# 4 Regression

In [6]:
for formula in ['NonLocal.Day', 'NonLocal.Night', 'HousePrice']:
    feature_name = f'GwrModel_2021_Merged_{formula}'
    in_json_file = JoinPath(root, 'Results', f'{feature_name}.geojson')
    out_features = JoinPath(root, 'Gwr.gdb', feature_name.replace('.', '_'))
    with arcpy.EnvManager(addOutputsToMap = False):
        arcpy.conversion.JSONToFeatures(
            in_json_file = in_json_file,
            out_features = out_features,
            geometry_type = 'POLYGON'
        )

In [7]:
for quarter in range(1, 5):
    for formula in ['NonLocal.Day', 'NonLocal.Night', 'HousePrice']:
        feature_name = f'GwrModel_2021Q{quarter}_{formula}'
        in_json_file = JoinPath(root, 'Results', f'{feature_name}.geojson')
        out_features = JoinPath(root, 'Gwr.gdb', feature_name.replace('.', '_'))
        with arcpy.EnvManager(addOutputsToMap = False):
            arcpy.conversion.JSONToFeatures(
                in_json_file = in_json_file,
                out_features = out_features,
                geometry_type = 'POLYGON'
            )